In [ ]:
# Import required libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import shufflenet_v2_x1_0
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from PIL import Image
import numpy as np

# 1. Load and Prepare the Dataset
# Define transforms for your dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Use ImageFolder to load your custom dataset (change the path to your dataset)
train_dataset = datasets.ImageFolder(root=r'F:\ABDUL\ABDUL 2024\BRAIN_DAMAGE\Brain-UI\train', transform=transform)
val_dataset = datasets.ImageFolder(root=r'F:\ABDUL\ABDUL 2024\BRAIN_DAMAGE\Brain-UI\val', transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 2. Initialize ShuffleNet Model
num_classes = len(train_dataset.classes)  # Get the number of classes
model = shufflenet_v2_x1_0(pretrained=True)  # Load pre-trained ShuffleNet
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer for our dataset

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 3. Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Initialize lists to store the loss values
train_losses = []
val_losses = []

# 4. Training Loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation Loop
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()

    # Calculate average validation loss
    avg_val_loss = val_running_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}')

# 5. Plot Training and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='orange')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

# 6. Classification Report
# Gather predictions and true labels
all_predictions = []
all_labels = []

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Classification Report
report = classification_report(all_labels, all_predictions, target_names=train_dataset.classes)
print(report)

# 7. Confusion Matrix
cm = confusion_matrix(all_labels, all_predictions)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.classes)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# 8. Prediction Function
def predict(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path)  # Open the image
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
    predicted_class = class_names[predicted.item()]  # Get the class name
    return predicted_class

# Example usage of prediction
image_path = r'F:\ABDUL\ABDUL 2024\BRAIN_DAMAGE\Brain-UI\train\glioma_tumor\image(60).jpg'  # Path to an image
predicted_class = predict(image_path, model, transform, train_dataset.classes)
print(f'Predicted Class: {predicted_class}')

# 9. Display the Image with the Predicted Class
def display_image_with_prediction(image_path, predicted_class):
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(f'Predicted Class: {predicted_class}')
    plt.axis('off')
    plt.show()

# Display the image with its predicted class
display_image_with_prediction(image_path, predicted_class)
